In [2]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from pathlib import Path
from rasterio.warp import reproject
from rasterio.io import MemoryFile
from rasterio.merge import merge

In [3]:
def load_landsat_image(img_folder, bands):
    image = {}
    ds = {}
    path = Path(img_folder)
    for band in bands:
        # considering the landsat images end with *_SR_B#.TIF, we will use it to locate the correct file
        file = next(path.glob(f'*_T1_{band}.TIF'))
        print(f'Opening file {file}')
        ds = rasterio.open(file)
        image.update({band: ds.read(1)})

    return image, ds

In [4]:
# load the image
img, image_ds = load_landsat_image('/home/skalena/Área de trabalho/class_area/2022.08.08', ['B2', 'B3', 'B4', 'B5', 'B6','B7','B8', 'QA_PIXEL'])



Opening file /home/skalena/Área de trabalho/class_area/2022.08.08/LC08_L1TP_228061_20220808_20220818_02_T1_B2.TIF
Opening file /home/skalena/Área de trabalho/class_area/2022.08.08/LC08_L1TP_228061_20220808_20220818_02_T1_B3.TIF
Opening file /home/skalena/Área de trabalho/class_area/2022.08.08/LC08_L1TP_228061_20220808_20220818_02_T1_B4.TIF
Opening file /home/skalena/Área de trabalho/class_area/2022.08.08/LC08_L1TP_228061_20220808_20220818_02_T1_B5.TIF
Opening file /home/skalena/Área de trabalho/class_area/2022.08.08/LC08_L1TP_228061_20220808_20220818_02_T1_B6.TIF
Opening file /home/skalena/Área de trabalho/class_area/2022.08.08/LC08_L1TP_228061_20220808_20220818_02_T1_B7.TIF
Opening file /home/skalena/Área de trabalho/class_area/2022.08.08/LC08_L1TP_228061_20220808_20220818_02_T1_B8.TIF
Opening file /home/skalena/Área de trabalho/class_area/2022.08.08/LC08_L1TP_228061_20220808_20220818_02_T1_QA_PIXEL.TIF


In [5]:
# create the RGB for the image
rgb = np.stack([img['B4'], img['B3'], img['B2']], axis=2)
rgb = rgb / rgb.max()

In [6]:
# load the water masks
water1_ds = rasterio.open('/home/skalena/Área de trabalho/class_area/data/occurrence_70W_0Nv1_4_2021.tif')


water1_array = water1_ds.read(1)


In [7]:
water2_ds = rasterio.open('/home/skalena/Área de trabalho/class_area/data/occurrence_70W_0Nv1_4_2021.tif')
water2_array = water2_ds.read(1)

In [23]:
# Plot the 3 images
'''fig, ax = plt.subplots(1, figsize=(20, 6))
ax[0].imshow(rgb*3)
ax[1].imshow(water1_array)
ax[2].imshow(water2_array)'''

'fig, ax = plt.subplots(1, figsize=(20, 6))\nax[0].imshow(rgb*3)\nax[1].imshow(water1_array)\nax[2].imshow(water2_array)'

In [8]:
print('Image: \n', image_ds['B2'].profile)
print('Water mask 1: \n', water1_ds.profile)
print('Water mask 2: \n', water2_ds.profile)

TypeError: 'DatasetReader' object is not subscriptable

In [35]:
def create_dataset(data, crs, transform):
    # Receives a 2D array, a transform and a crs to create a rasterio dataset
    memfile = MemoryFile()
    dataset = memfile.open(driver='GTiff', height=data.shape[0], width=data.shape[1], count=1, crs=crs, transform=transform, dtype=data.dtype)
    dataset.write(data, 1)
        
    return dataset
